In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pyspark.sql.functions import col, count, isnan, when


In [0]:
# Load the dataset into a DataFrame
file_path = "dbfs:/user/hive/warehouse/air_quality_uci" 

df = spark.read.format("delta").load(file_path)


In [0]:
def calculate_data_quality_metrics(selected_column):
    total_rows = df.count()
    missing_values = df.where(col(selected_column).isNull() | isnan(col(selected_column))).count()
    percentage_missing = (missing_values / total_rows) * 100

    return total_rows, missing_values, percentage_missing


In [0]:
pip install plotly

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import pandas as pd
import pyspark.sql.functions as F
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact

# Load the dataset into a DataFrame
file_path = "dbfs:/user/hive/warehouse/air_quality_uci" 
df = spark.read.format("delta").load(file_path)

def calculate_data_quality_metrics(selected_column):
    total_rows = df.count()

    # Handle column name with special characters using backticks
    if selected_column == 'Date':
        missing_values = df.where(F.col(selected_column).isNull()).count()
    else:
        missing_values = df.where(F.col("`{}`".format(selected_column)).isNull() | F.isnan(F.col("`{}`".format(selected_column)))).count()

    percentage_missing = (missing_values / total_rows) * 100

    return total_rows, missing_values, percentage_missing

columns_list = df.columns

@interact(selected_column=widgets.Dropdown(options=columns_list, description='Select Column:'))
def update_dashboard(selected_column):
    total_rows, missing_values, percentage_missing = calculate_data_quality_metrics(selected_column)

    # Data quality metrics table
    data = {
        'Total Rows': [total_rows],
        'Missing Values': [missing_values],
        'Percentage Missing': [percentage_missing],
    }
    df_metrics = pd.DataFrame(data)

    # Bar chart for missing values count
    plt.figure(figsize=(8, 4))
    plt.bar(df_metrics.columns, df_metrics.iloc[0])
    plt.xlabel('Metrics')
    plt.ylabel('Count')
    plt.title('Data Quality Metrics')
    plt.show()

    # Display the data quality metrics table
    display(df_metrics)





interactive(children=(Dropdown(description='Select Column:', options=('Date', 'Time', 'CO(GT)', 'PT08.S1(CO)',…